In [ ]:
import pandas as pd
import numpy as np
import pymysql
import pyodbc
import hashlib
import json
from datetime import datetime, timedelta
import random
from pymongo import MongoClient

# Función principal
def main():
    print("Iniciando proceso de migración de datos a Soltura...")
    
    # 1. Extraer datos del sistema fuente
    print("Extrayendo usuarios...")
    df_users = extract_users_from_source()
    if df_users is None:
        print("Error: No se pudieron extraer los usuarios. Abortando.")
        return
    
    print(f"Se encontraron {len(df_users)} usuarios para migrar.")
    
    print("Extrayendo planes...")
    df_plans = extract_plans_from_source()
    if df_plans is None:
        print("Error: No se pudieron extraer los planes. Abortando.")
        return
    
    print(f"Se encontraron {len(df_plans)} planes para migrar.")
    
    print("Extrayendo permisos de usuarios...")
    user_permissions = extract_user_permissions()
    if user_permissions is None:
        print("Error: No se pudieron extraer los permisos. Abortando.")
        return
    
    print("Extrayendo información de contacto...")
    df_contact_info = extract_user_contact_info()
    if df_contact_info is None:
        print("Error: No se pudo extraer la información de contacto. Abortando.")
        return
    
    print("Extrayendo direcciones...")
    df_addresses = extract_user_addresses()
    if df_addresses is None:
        print("Error: No se pudieron extraer las direcciones. Abortando.")
        return
    
    print("Extrayendo métodos de pago...")
    df_payment_methods = extract_user_payment_methods()
    if df_payment_methods is None:
        print("Error: No se pudieron extraer los métodos de pago. Abortando.")
        return
    
    # 2. Mapear y migrar planes
    print("Mapeando planes a Soltura...")
    plan_mapping, new_plans = map_plans_to_soltura(df_plans)
    if plan_mapping is None:
        print("Error: No se pudieron mapear los planes. Abortando.")
        return
    
    print(f"Se mapearon {len(plan_mapping)} planes.")
    
    # 3. Migrar usuarios y sus suscripciones
    print("Migrando usuarios y suscripciones...")
    user_mapping = migrate_users_and_subscriptions(df_users, plan_mapping, df_contact_info, df_addresses)
    if user_mapping is None:
        print("Error: No se pudieron migrar los usuarios. Abortando.")
        return
    
    print(f"Se migraron {len(user_mapping)} usuarios.")
    
    # 4. Migrar permisos de usuarios
    print("Migrando permisos de usuarios...")
    success = migrate_user_permissions(user_permissions, user_mapping)
    if not success:
        print("Error: No se pudieron migrar los permisos.")
    
    # 5. Migrar métodos de pago
    print("Migrando métodos de pago...")
    success = migrate_payment_methods(df_payment_methods, user_mapping)
    if not success:
        print("Error: No se pudieron migrar los métodos de pago.")
    
    # 6. Crear banners de marketing
    print("Creando banners de marketing...")
    success = create_marketing_banners(mysql_config['database'])
    if not success:
        print("Error: No se pudieron crear los banners de marketing.")
    
    # 7. Generar emails de restablecimiento de contraseña
    print("Generando emails de restablecimiento de contraseña...")
    generate_password_reset_emails()
    
    # 8. Verificar estado final de la migración
    check_migration_status()
    
    print("\nProceso de migración completado.")
    
    # Resumen
    print("\n=== Resumen de la migración ===")
    print(f"Sistema origen: {mysql_config['database']}")
    print(f"Usuarios migrados: {len(user_mapping)}")
    print(f"Planes migrados: {len(plan_mapping)}")
    print("Nuevos planes creados:")
    for plan in new_plans:
        print(f"  - {plan['name']} (ID: {plan['soltura_plan_id']})")
    print("\nPara completar la migración, se deben enviar los emails de restablecimiento")
    print("de contraseña generados en 'reset_password_emails.json'")
    print("=============================")




def check_migration_status():
    """
    Verifica y muestra el estado de la migración:
    - Total de usuarios migrados
    - Distribución de usuarios por tipo de suscripción
    - Porcentaje de usuarios con credenciales reseteo pendiente
    """
    conn_sql = connect_sqlserver()
    if not conn_sql:
        return False
    
    try:
        cursor = conn_sql.cursor()
        
        # Verificar total de usuarios migrados
        cursor.execute("""
        SELECT COUNT(*) FROM SocaiUserMigrations
        WHERE SourceSystem = ?
        """, (mysql_config['database'],))
        
        total_migrated = cursor.fetchone()[0]
        
        # Verificar usuarios con reseteo de contraseña pendiente
        cursor.execute("""
        SELECT COUNT(*) FROM SocaiUserMigrations
        WHERE SourceSystem = ? AND ResetPassword = 1
        """, (mysql_config['database'],))
        
        reset_pending = cursor.fetchone()[0]
        
        # Verificar distribución de suscripciones
        cursor.execute("""
        SELECT s.Name AS PlanName, COUNT(su.SubscriptionUserId) AS UserCount
        FROM SocaiUserMigrations um
        JOIN SocaiUsers u ON um.SolturaUserId = u.UserId
        JOIN SocaiSubscriptionUser su ON u.UserId = su.UserId
        JOIN SocaiSubscriptions s ON su.SubscriptionId = s.SubscriptionId
        WHERE um.SourceSystem = ?
        GROUP BY s.Name
        ORDER BY UserCount DESC
        """, (mysql_config['database'],))
        
        subscription_distribution = cursor.fetchall()
        
        # Imprimir resultados
        print("\n=== ESTADO DE LA MIGRACIÓN ===")
        print(f"Total de usuarios migrados: {total_migrated}")
        if total_migrated > 0:
            print(f"Usuarios con reseteo de contraseña pendiente: {reset_pending} ({reset_pending/total_migrated*100:.2f}%)")
            
            print("\nDistribución de suscripciones:")
            for plan in subscription_distribution:
                print(f"  - {plan[0]}: {plan[1]} usuarios")
        
        return True
    except Exception as e:
        print(f"Error verificando estado de migración: {e}")
        return False
    finally:
        conn_sql.close()

# Añadir función para ayudar a usuarios a restablecer contraseñas
def generate_password_reset_emails():
    """
    Genera contenido para emails de restablecimiento de contraseña
    para los usuarios migrados
    """
    conn = connect_sqlserver()
    if not conn:
        return False
    
    try:
        cursor = conn.cursor()
        
        # Obtener usuarios con reseteo pendiente
        cursor.execute("""
        SELECT um.SolturaUserId, u.Email, u.Name, um.OriginalEmail
        FROM SocaiUserMigrations um
        JOIN SocaiUsers u ON um.SolturaUserId = u.UserId
        WHERE um.SourceSystem = ? AND um.ResetPassword = 1
        """, (mysql_config['database'],))
        
        users_to_reset = cursor.fetchall()
        
        # Crear contenido para emails
        reset_emails_content = []
        for user in users_to_reset:
            user_id = user[0]
            email = user[1]
            name = user[2]
            original_email = user[3] if user[3] else email
            
            # Generar token único para este usuario (simulado)
            reset_token = hashlib.sha256(f"{user_id}_{datetime.now()}".encode()).hexdigest()[:20]
            
            # Preparar contenido del email
            email_content = {
                "to": email,
                "subject": "Bienvenido a Soltura - Establece tu nueva contraseña",
                "body": f"""
                <html>
                <body>
                    <h2>Hola {name},</h2>
                    
                    <p>¡Bienvenido a Soltura! Tu cuenta de {mysql_config['database']} ha sido migrada exitosamente.</p>
                    
                    <p>Como parte del proceso de migración, necesitas establecer una nueva contraseña para acceder a todos tus beneficios, que ahora incluyen dos servicios adicionales sin costo extra.</p>
                    
                    <p>Para establecer tu contraseña, haz clic en el siguiente enlace:</p>
                    
                    <p><a href="https://soltura.com/reset-password?token={reset_token}&email={email}">Establecer mi nueva contraseña</a></p>
                    
                    <p>Este enlace expirará en 48 horas.</p>
                    
                    <p>Si no solicitaste este cambio o tienes alguna pregunta, por favor contáctanos respondiendo a este correo o llamando a nuestro servicio al cliente.</p>
                    
                    <p>Saludos,<br>
                    El equipo de Soltura</p>
                </body>
                </html>
                """
            }
            
            reset_emails_content.append(email_content)
        
        # Guardar los emails en un archivo JSON para procesamiento posterior
        with open("reset_password_emails.json", "w", encoding="utf-8") as f:
            json.dump(reset_emails_content, f, ensure_ascii=False, indent=4)
            
        print(f"Se generaron {len(reset_emails_content)} emails de restablecimiento de contraseña.")
        print("Los contenidos se guardaron en 'reset_password_emails.json' para su envío.")
        
        return True
    except Exception as e:
        print(f"Error generando emails de restablecimiento: {e}")
        return False
    finally:
        conn.close()# Función para crear los banners de marketing en MongoDB
def create_marketing_banners(app_name):
    """
    Crea los banners de marketing en MongoDB
    """
    db = connect_mongodb()
    if not db:
        return False
    
    try:
        # Colección para banners de marketing
        collection = db["marketing_banners"]
        
        # Fecha de migración
        migration_date = datetime.now().strftime("%d/%m/%Y")
        
        # Banner para home page
        home_banner = {
            "type": "home_image",
            "title": f"{app_name} ahora es Soltura",
            "content": f"{app_name} se ha unido a la familia Soltura para ofrecerte más beneficios con los mismos precios.",
            "migration_date": migration_date,
            "guide_link": "https://soltura.com/migracion-guia",
            "image_url": "https://soltura.com/images/migration_banner.jpg",
            "active": True,
            "created_at": datetime.now(),
            "show_until": datetime.now() + timedelta(days=30)
        }
        
        # Banner publicitario
        promo_banner = {
            "type": "promotional_banner",
            "title": f"¡Bienvenido a Soltura!",
            "content": f"Como usuario de {app_name}, ahora disfrutas de todos tus beneficios más 2 servicios adicionales con Soltura.",
            "migration_date": migration_date,
            "guide_link": "https://soltura.com/migracion-guia",
            "button_text": "Más información",
            "active": True,
            "created_at": datetime.now(),
            "show_until": datetime.now() + timedelta(days=60),
            "target_audience": "migrated_users"
        }
        
        # Insertar banners
        collection.insert_many([home_banner, promo_banner])
        
        print(f"Banners de marketing creados en MongoDB para anunciar la migración de {app_name} a Soltura.")
        return True
    except Exception as e:
        print(f"Error creando banners: {e}")
        return "Falseimport pandas as pd"

# Configuración de conexión a la base de datos fuente (MySQL) - Payment Assistant
mysql_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'password',
    'database': 'database'  
}

# Configuración de conexión a la base de datos destino (SQL Server) - Soltura
sqlserver_config = {
    'driver': '{ODBC Driver 17 for SQL Server}',
    'server': 'localhost',
    'database': 'Caso2',
    'trusted_connection': 'yes'
}

# Configuración de MongoDB para las colecciones de marketing
mongo_config = {
    'host': 'localhost',
    'port': 27017,
    'database': 'soltura_marketing'
}

# Función para conectarse a MySQL
def connect_mysql():
    try:
        conn = pymysql.connect(
            host=mysql_config['host'],
            user=mysql_config['user'],
            password=mysql_config['password'],
            database=mysql_config['database']
        )
        return conn
    except Exception as e:
        print(f"Error conectando a MySQL: {e}")
        return None

# Función para conectarse a SQL Server
def connect_sqlserver():
    try:
        conn = pyodbc.connect(
            f"DRIVER={sqlserver_config['driver']};"
            f"SERVER={sqlserver_config['server']};"
            f"DATABASE={sqlserver_config['database']};"
            f"Trusted_Connection={sqlserver_config['trusted_connection']};"
        )
        return conn
    except Exception as e:
        print(f"Error conectando a SQL Server: {e}")
        return None

# Función para conectarse a MongoDB
def connect_mongodb():
    try:
        client = MongoClient(mongo_config['host'], mongo_config['port'])
        db = client[mongo_config['database']]
        return db
    except Exception as e:
        print(f"Error conectando a MongoDB: {e}")
        return None

# Función para extraer usuarios de Payment Assistant
def extract_users_from_source():
    conn = connect_mysql()
    if not conn:
        return None
    
    try:
        # Usamos la estructura real de las tablas de Payment Assistant
        query = """
        SELECT u.userid, u.name, ui.value as email, ua.addressid, u.fecha_registro, u.isactive,
               us.usersubsid, us.plansid, us.start_date, us.end_date, us.status, us.autorenew,
               pp.billingperiod as payment_frequency, pp.price, c.acronym as currency
        FROM users u
        LEFT JOIN contactinfoperperson ui ON u.userid = ui.userid
        LEFT JOIN contactinfotype ct ON ui.contactinfotypeid = ct.contactinfotypeid AND ct.name = 'Email'
        LEFT JOIN useraddresses ua ON u.userid = ua.userid
        LEFT JOIN userssubscriptions us ON u.userid = us.userid
        LEFT JOIN planpricing pp ON us.priceid = pp.priceid
        LEFT JOIN currencies c ON pp.currencyId = c.currencyId
        WHERE u.isactive = 1
        """
        
        df_users = pd.read_sql(query, conn)
        return df_users
    except Exception as e:
        print(f"Error extrayendo usuarios: {e}")
        return None
    finally:
        conn.close()

# Función para extraer planes de Payment Assistant
def extract_plans_from_source():
    conn = connect_mysql()
    if not conn:
        return None
    
    try:
        query = """
        SELECT p.plansid, p.name, p.description, p.isactive, p.createdat, p.updatedat,
               pp.priceid, pp.billingperiod, pp.price, c.acronym as currency,
               GROUP_CONCAT(f.name) as features
        FROM plans p
        JOIN planpricing pp ON p.plansid = pp.plansid
        JOIN currencies c ON pp.currencyId = c.currencyId
        LEFT JOIN planfeatures pf ON p.plansid = pf.plansid
        LEFT JOIN features f ON pf.featureid = f.featureid
        WHERE p.isactive = 1
        GROUP BY p.plansid, pp.priceid
        """
        
        df_plans = pd.read_sql(query, conn)
        return df_plans
    except Exception as e:
        print(f"Error extrayendo planes: {e}")
        return None
    finally:
        conn.close()

# Función para extraer permisos de usuarios de Payment Assistant
def extract_user_permissions():
    conn = connect_mysql()
    if not conn:
        return None
    
    try:
        query = """
        SELECT up.userid, up.permissionid, p.description, p.code, m.name as module_name
        FROM userspermissions up
        JOIN permissions p ON up.permissionid = p.permissionid
        JOIN modules m ON p.moduleid = m.moduleid
        WHERE up.enabled = 1 AND up.deleted = 0
        """
        
        df_permissions = pd.read_sql(query, conn)
        
        # También extraemos los roles de los usuarios
        query_roles = """
        SELECT ur.userid, ur.roleid, r.name as role_name
        FROM usersroles ur
        JOIN roles r ON ur.roleid = r.roleid
        WHERE ur.enabled = 1 AND ur.deleted = 0
        """
        
        df_roles = pd.read_sql(query_roles, conn)
        
        return {'permissions': df_permissions, 'roles': df_roles}
    except Exception as e:
        print(f"Error extrayendo permisos y roles: {e}")
        return None
    finally:
        conn.close()

# Función para extraer información de contacto adicional
def extract_user_contact_info():
    conn = connect_mysql()
    if not conn:
        return None
    
    try:
        query = """
        SELECT cip.userid, cit.name as contact_type, cip.value
        FROM contactinfoperperson cip
        JOIN contactinfotype cit ON cip.contactinfotypeid = cit.contactinfotypeid
        WHERE cip.enabled = 1
        """
        
        df_contact_info = pd.read_sql(query, conn)
        return df_contact_info
    except Exception as e:
        print(f"Error extrayendo información de contacto: {e}")
        return None
    finally:
        conn.close()

# Función para extraer direcciones de usuarios
def extract_user_addresses():
    conn = connect_mysql()
    if not conn:
        return None
    
    try:
        query = """
        SELECT ua.userid, a.addressid, a.line1, a.line2, a.zipcode, 
               c.name as city, s.name as state, co.name as country
        FROM useraddresses ua
        JOIN addresses a ON ua.addressid = a.addressid
        JOIN city c ON a.cityid = c.cityid
        JOIN state s ON c.stateid = s.stateid
        JOIN country co ON s.countryid = co.countryid
        WHERE ua.enabled = 1
        """
        
        df_addresses = pd.read_sql(query, conn)
        return df_addresses
    except Exception as e:
        print(f"Error extrayendo direcciones: {e}")
        return None
    finally:
        conn.close()

# Función para extraer métodos de pago de usuarios
def extract_user_payment_methods():
    conn = connect_mysql()
    if not conn:
        return None
    
    try:
        query = """
        SELECT upm.userid, upm.userpaymentmethodid, pm.name as payment_method,
               upm.accountinfo, upm.accountnumber, upm.isdefault
        FROM userpaymentmethods upm
        JOIN paymentmethods pm ON upm.paymentmethodid = pm.paymentmethodid
        WHERE upm.enabled = 1
        """
        
        df_payment_methods = pd.read_sql(query, conn)
        return df_payment_methods
    except Exception as e:
        print(f"Error extrayendo métodos de pago: {e}")
        return None
    finally:
        conn.close()

# Función para mapear planes de Payment Assistant a planes de Soltura
def map_plans_to_soltura(df_plans):
    """
    Mapea los planes del sistema fuente a los planes de Soltura
    Se crean nuevos planes en Soltura que incluyen lo del plan original más dos beneficios adicionales
    """
    conn = connect_sqlserver()
    if not conn:
        return None, None
    
    try:
        # Obtener planes actuales de Soltura
        cursor = conn.cursor()
        cursor.execute("SELECT SubscriptionId, Name, Description, amount, CurrencyTypeId FROM SocaiSubscriptions")
        soltura_plans = cursor.fetchall()
        
        # Mapear planes
        plan_mapping = {}
        new_plans = []
        
        for idx, plan in df_plans.iterrows():
            # Crear un nuevo plan en Soltura basado en el plan original
            new_plan_name = f"Migrado {plan['name']} Plus"
            new_plan_desc = f"Plan migrado desde {mysql_config['database']} con beneficios adicionales. {plan['description']}"
            
            # Los planes migrados serán personalizables
            is_customizable = 1
            
            # El monto será similar al original pero ajustado al tipo de moneda de Soltura
            amount = float(plan['price'])
            
            # Determinamos el CurrencyTypeId basado en la moneda del plan original
            currency_type_id = 3  # Default: CRC
            if plan['currency'] == 'USD':
                currency_type_id = 1
            elif plan['currency'] == 'EUR':
                currency_type_id = 2
            
            # Insertar el nuevo plan en Soltura
            sql = """
            INSERT INTO SocaiSubscriptions (Name, Description, isCustomizable, isActive, createdAt, updatedAt, amount, CurrencyTypeId)
            VALUES (?, ?, ?, 1, GETDATE(), GETDATE(), ?, ?)
            """
            cursor.execute(sql, (new_plan_name, new_plan_desc, is_customizable, amount, currency_type_id))
            
            # Obtener el ID del plan recién insertado
            cursor.execute("SELECT @@IDENTITY")
            new_plan_id = cursor.fetchone()[0]
            
            # Guardar la relación entre planes originales y nuevos
            plan_mapping[plan['plansid']] = new_plan_id
            new_plans.append({
                'source_plan_id': plan['plansid'],
                'soltura_plan_id': new_plan_id,
                'name': new_plan_name
            })
            
            # Agregar beneficios al nuevo plan
            # Procesar las características originales que vienen como cadena separada por comas
            if pd.notna(plan['features']):
                original_features = plan['features'].split(',')
            else:
                original_features = []
            
            # Insertamos los beneficios originales
            for feature in original_features:
                # Buscamos una característica existente similar en Soltura
                cursor.execute("SELECT TOP 1 FeatureId FROM SocaiPlanFeatures WHERE Name LIKE ?", (f"%{feature.strip()}%",))
                result = cursor.fetchone()
                
                if result:
                    feature_id = result[0]
                    
                    # Determinamos un tipo de unidad apropiado para esta característica
                    cursor.execute("SELECT UnitTypeId FROM SocaiPlanFeatures WHERE FeatureId = ?", (feature_id,))
                    unit_type_id = cursor.fetchone()[0]
                    
                    # Determinamos un tipo de servicio apropiado
                    cursor.execute("SELECT TOP 1 ServiceTypeId FROM SocaiServiceTypes WHERE Name LIKE ?", (f"%{feature.strip()}%",))
                    service_type_result = cursor.fetchone()
                    
                    if service_type_result:
                        service_type_id = service_type_result[0]
                    else:
                        # Si no encontramos uno específico, seleccionamos uno aleatorio
                        cursor.execute("SELECT TOP 1 ServiceTypeId FROM SocaiServiceTypes ORDER BY NEWID()")
                        service_type_id = cursor.fetchone()[0]
                    
                    # Insertamos la relación plan-característica
                    sql = """
                    INSERT INTO SocaiFeaturesSubscriptions 
                    (PlanFeatureId, SubscriptionId, Quantity, UnitTypeId, CreatedAt, UpdatedAt, ServiceTypeId, MemberCount, IsMemberSpecific)
                    VALUES (?, ?, 1.0, ?, GETDATE(), GETDATE(), ?, 1, 0)
                    """
                    cursor.execute(sql, (feature_id, new_plan_id, unit_type_id, service_type_id))
            
            # Agregar dos beneficios adicionales (como prometió Soltura)
            # Seleccionamos 2 características aleatorias que no estén ya en el plan
            cursor.execute("""
            SELECT TOP 2 f.FeatureId, f.UnitTypeId 
            FROM SocaiPlanFeatures f
            WHERE f.FeatureId NOT IN (
                SELECT fs.PlanFeatureId 
                FROM SocaiFeaturesSubscriptions fs 
                WHERE fs.SubscriptionId = ?
            )
            ORDER BY NEWID()
            """, (new_plan_id,))
            
            additional_features = cursor.fetchall()
            
            # Para cada característica adicional
            for feature in additional_features:
                feature_id = feature[0]
                unit_type_id = feature[1]
                
                # Seleccionamos un tipo de servicio apropiado
                cursor.execute("SELECT TOP 1 ServiceTypeId FROM SocaiServiceTypes ORDER BY NEWID()")
                service_type_id = cursor.fetchone()[0]
                
                # Insertamos la relación con un valor mayor para destacar que es un beneficio adicional
                sql = """
                INSERT INTO SocaiFeaturesSubscriptions 
                (PlanFeatureId, SubscriptionId, Quantity, UnitTypeId, CreatedAt, UpdatedAt, ServiceTypeId, MemberCount, IsMemberSpecific)
                VALUES (?, ?, 2.0, ?, GETDATE(), GETDATE(), ?, 1, 0)
                """
                cursor.execute(sql, (feature_id, new_plan_id, unit_type_id, service_type_id))
        
        conn.commit()
        return plan_mapping, new_plans
    except Exception as e:
        conn.rollback()
        print(f"Error mapeando planes: {e}")
        return None, None
    finally:
        conn.close()

# Función para migrar usuarios y sus suscripciones
def migrate_users_and_subscriptions(df_users, plan_mapping, df_contact_info, df_addresses):
    """
    Migra los usuarios y sus suscripciones a Soltura utilizando los datos reales de Payment Assistant
    """
    conn = connect_sqlserver()
    if not conn:
        return None
    
    try:
        cursor = conn.cursor()
        
        # Crear tabla para mapeo de usuarios si no existe
        cursor.execute("""
        IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'SocaiUserMigrations')
        BEGIN
            CREATE TABLE SocaiUserMigrations (
                MigrationId INT IDENTITY(1,1) PRIMARY KEY,
                SourceUserId INT NOT NULL,
                SolturaUserId INT NOT NULL,
                SourceSystem VARCHAR(50) NOT NULL,
                MigrationDate DATETIME NOT NULL,
                ResetPassword BIT NOT NULL DEFAULT 1,
                OriginalEmail VARCHAR(255) NULL
            )
        END
        """)
        
        # Para cada usuario
        user_mapping = {}
        active_subscriptions = 0
        annual_subscriptions = 0
        
        # Agrupar usuarios por userid para evitar duplicados
        grouped_users = df_users.groupby('userid').first().reset_index()
        
        for idx, user in grouped_users.iterrows():
            # Verificamos si el usuario ya existe en Soltura (por email)
            if pd.notna(user['email']):
                cursor.execute("SELECT UserId FROM SocaiUsers WHERE Email = ?", (user['email'],))
                existing_user = cursor.fetchone()
            else:
                existing_user = None
            
            if existing_user:
                # Si existe, lo mapeamos
                user_mapping[user['userid']] = existing_user[0]
                
                # Registramos el mapeo
                cursor.execute("""
                INSERT INTO SocaiUserMigrations (SourceUserId, SolturaUserId, SourceSystem, MigrationDate, ResetPassword, OriginalEmail)
                VALUES (?, ?, ?, GETDATE(), 1, ?)
                """, (user['userid'], existing_user[0], mysql_config['database'], user['email']))
            else:
                # Si no existe, creamos un nuevo usuario
                # Obtenemos la dirección del usuario si está disponible
                address_info = df_addresses[df_addresses['userid'] == user['userid']].iloc[0] if not df_addresses[df_addresses['userid'] == user['userid']].empty else None
                
                # Seleccionamos una dirección aleatoria existente en Soltura o creamos una nueva
                if address_info is not None:
                    # Buscar una ciudad similar en Soltura
                    cursor.execute("SELECT TOP 1 CityID FROM SocaiCities WHERE Name LIKE ?", (f"%{address_info['city']}%",))
                    city_result = cursor.fetchone()
                    
                    if city_result:
                        city_id = city_result[0]
                    else:
                        # Si no encontramos una ciudad similar, seleccionamos una aleatoria
                        cursor.execute("SELECT TOP 1 CityID FROM SocaiCities ORDER BY NEWID()")
                        city_id = cursor.fetchone()[0]
                    
                    # Creamos una nueva dirección
                    address_line = f"{address_info['line1']}, {address_info['line2'] if pd.notna(address_info['line2']) else ''}"
                    postal_code = address_info['zipcode'] if pd.notna(address_info['zipcode']) else "10101"
                    
                    cursor.execute("""
                    INSERT INTO SocaiAdresses (PostalCode, CityId, direccion, CreatedAt, UpdatedAt, point)
                    VALUES (?, ?, ?, GETDATE(), GETDATE(), NULL)
                    """, (postal_code, city_id, address_line))
                    
                    cursor.execute("SELECT @@IDENTITY")
                    address_id = cursor.fetchone()[0]
                else:
                    # Seleccionamos una dirección aleatoria existente
                    cursor.execute("SELECT TOP 1 AddressId FROM SocaiAdresses ORDER BY NEWID()")
                    address_id = cursor.fetchone()[0]
                
                # Obtenemos información de contacto adicional
                phone = None
                if not df_contact_info[df_contact_info['userid'] == user['userid']].empty:
                    phone_info = df_contact_info[(df_contact_info['userid'] == user['userid']) & 
                                              (df_contact_info['contact_type'] == 'Phone')]
                    if not phone_info.empty:
                        phone = phone_info.iloc[0]['value']
                
                if phone is None:
                    phone = "8000-0000"  # Valor por defecto
                
                # Generamos un hash temporal como contraseña
                temp_password = hashlib.sha256(f"{user['email']}_{datetime.now()}".encode()).digest()
                
                # Insertamos el nuevo usuario
                sql = """
                INSERT INTO SocaiUsers (Name, Email, PhoneNumber, Password, AddressId, isActive, LastLogin, CreatedAt)
                VALUES (?, ?, ?, ?, ?, 1, GETDATE(), ?)
                """
                
                # Usamos los datos del usuario original o valores predeterminados si faltan
                name = user['name'] if pd.notna(user['name']) else f"Usuario {user['userid']}"
                email = user['email'] if pd.notna(user['email']) else f"usuario{user['userid']}@migracion.soltura.cr"
                created_at = user['fecha_registro'] if pd.notna(user['fecha_registro']) else datetime.now()
                
                cursor.execute(sql, (name, email, phone, temp_password, address_id, created_at))
                
                # Obtenemos el ID del usuario recién insertado
                cursor.execute("SELECT @@IDENTITY")
                new_user_id = cursor.fetchone()[0]
                
                # Guardamos el mapeo
                user_mapping[user['userid']] = new_user_id
                
                # Registramos el mapeo en la tabla de migraciones
                cursor.execute("""
                INSERT INTO SocaiUserMigrations (SourceUserId, SolturaUserId, SourceSystem, MigrationDate, ResetPassword, OriginalEmail)
                VALUES (?, ?, ?, GETDATE(), 1, ?)
                """, (user['userid'], new_user_id, mysql_config['database'], email))
                
                # Si el usuario tiene una suscripción activa, la migramos
                if pd.notna(user['usersubsid']) and pd.notna(user['plansid']) and user['plansid'] in plan_mapping:
                    # Obtenemos el ID del plan en Soltura
                    soltura_plan_id = plan_mapping[user['plansid']]
                    
                    # Determinamos las fechas de inicio y fin
                    if pd.notna(user['start_date']):
                        start_date = user['start_date']
                    else:
                        start_date = datetime.now() - timedelta(days=random.randint(1, 15))
                    
                    if pd.notna(user['end_date']):
                        end_date = user['end_date']
                    else:
                        # Si es plan mensual, la fecha de fin es 30 días después del inicio
                        # Si es plan anual, la fecha de fin es 365 días después del inicio
                        if user['payment_frequency'] == 'monthly':
                            end_date = start_date + timedelta(days=30)
                            active_subscriptions += 1
                        else:  # anual
                            end_date = start_date + timedelta(days=365)
                            annual_subscriptions += 1
                            active_subscriptions += 1
                    
                    # Insertamos la suscripción
                    sql = """
                    INSERT INTO SocaiSubscriptionUser (enable, startDateTime, endDateTime, UserId, SubscriptionId)
                    VALUES (1, ?, ?, ?, ?)
                    """
                    cursor.execute(sql, (start_date, end_date, new_user_id, soltura_plan_id))
        
        conn.commit()
        print(f"Migración completa: {len(user_mapping)} usuarios migrados.")
        print(f"Suscripciones activas: {active_subscriptions}")
        print(f"Suscripciones anuales: {annual_subscriptions}")
        print(f"Suscripciones mensuales: {active_subscriptions - annual_subscriptions}")
        return user_mapping
    except Exception as e:
        conn.rollback()
        print(f"Error migrando usuarios: {e}")
        return None
    finally:
        conn.close()

# Función para migrar permisos de usuarios
def migrate_user_permissions(user_permissions, user_mapping):
    """
    Migra los permisos de los usuarios a Soltura
    """
    conn = connect_sqlserver()
    if not conn:
        return False
    
    try:
        cursor = conn.cursor()
        
        # Obtener los permisos y roles
        df_permissions = user_permissions['permissions']
        df_roles = user_permissions['roles']
        
        # Para cada usuario con permisos
        permission_count = 0
        role_count = 0
        
        # Migrar roles primero
        for idx, role in df_roles.iterrows():
            # Verificar si el usuario existe en el mapeo
            if role['userid'] in user_mapping:
                soltura_user_id = user_mapping[role['userid']]
                
                # Mapear el rol a un rol en Soltura
                role_name = role['role_name'] if pd.notna(role['role_name']) else "Unknown Role"
                
                # Buscar rol similar en Soltura
                cursor.execute("SELECT RoleId FROM SocaiRoles WHERE Name LIKE ?", (f"%{role_name}%",))
                role_result = cursor.fetchone()
                
                if role_result:
                    role_id = role_result[0]
                else:
                    # Si no encontramos un rol similar, usamos el rol "Cliente" o "Usuario" por defecto
                    cursor.execute("SELECT RoleId FROM SocaiRoles WHERE Name LIKE '%Cliente%' OR Name LIKE '%Usuario%'")
                    default_role = cursor.fetchone()
                    
                    if default_role:
                        role_id = default_role[0]
                    else:
                        # Si no encontramos ni siquiera un rol por defecto, continuamos con el siguiente
                        continue
                
                # Verificar si ya tiene este rol
                cursor.execute("""
                SELECT COUNT(*) FROM SocaiUserRoles 
                WHERE UserId = ? AND RoleId = ?
                """, (soltura_user_id, role_id))
                
                exists = cursor.fetchone()[0] > 0
                
                if not exists:
                    # Insertar el rol del usuario
                    sql = """
                    INSERT INTO SocaiUserRoles (UserId, RoleId, UpdatedAt)
                    VALUES (?, ?, GETDATE())
                    """
                    cursor.execute(sql, (soltura_user_id, role_id))
                    role_count += 1
        
        # Ahora migrar permisos específicos
        for idx, perm in df_permissions.iterrows():
            # Verificar si el usuario existe en el mapeo
            if perm['userid'] in user_mapping:
                soltura_user_id = user_mapping[perm['userid']]
                
                # Mapear el permiso a un permiso en Soltura
                perm_code = perm['code'] if pd.notna(perm['code']) else "Unknown"
                perm_desc = perm['description'] if pd.notna(perm['description']) else "Unknown Permission"
                
                # Buscar permiso similar en Soltura
                cursor.execute("SELECT PermissionID FROM SocaiPermissions WHERE Description LIKE ? OR Name LIKE ?", 
                               (f"%{perm_desc}%", f"%{perm_code}%"))
                perm_result = cursor.fetchone()
                
                if perm_result:
                    permission_id = perm_result[0]
                    
                    # Verificar si el usuario ya tiene asignado este permiso a través de algún rol
                    cursor.execute("""
                    SELECT COUNT(*) FROM SocaiRolePermissions rp
                    JOIN SocaiUserRoles ur ON rp.RoleId = ur.RoleId
                    WHERE ur.UserId = ? AND rp.PermissionID = ?
                    """, (soltura_user_id, permission_id))
                    
                    has_permission_via_role = cursor.fetchone()[0] > 0
                    
                    if not has_permission_via_role:
                        # Al parecer, Soltura no tiene una tabla directa de permisos por usuario
                        # Necesitamos asignar un rol que tenga este permiso
                        cursor.execute("""
                        SELECT RoleId FROM SocaiRolePermissions
                        WHERE PermissionID = ?
                        """, (permission_id,))
                        
                        role_with_permission = cursor.fetchone()
                        
                        if role_with_permission:
                            role_id = role_with_permission[0]
                            
                            # Verificar si ya tiene este rol
                            cursor.execute("""
                            SELECT COUNT(*) FROM SocaiUserRoles 
                            WHERE UserId = ? AND RoleId = ?
                            """, (soltura_user_id, role_id))
                            
                            exists = cursor.fetchone()[0] > 0
                            
                            if not exists:
                                # Insertar el rol del usuario
                                sql = """
                                INSERT INTO SocaiUserRoles (UserId, RoleId, UpdatedAt)
                                VALUES (?, ?, GETDATE())
                                """
                                cursor.execute(sql, (soltura_user_id, role_id))
                                permission_count += 1
        
        conn.commit()
        print(f"Migración de permisos completa: {role_count} roles y {permission_count} permisos migrados.")
        return True
    except Exception as e:
        conn.rollback()
        print(f"Error migrando permisos: {e}")
        return False
    finally:
        conn.close()

# Función para migrar métodos de pago
def migrate_payment_methods(df_payment_methods, user_mapping):
    """
    Migra los métodos de pago de los usuarios a Soltura
    """
    conn = connect_sqlserver()
    if not conn:
        return False
    
    try:
        cursor = conn.cursor()
        payment_methods_count = 0
        
        for idx, payment in df_payment_methods.iterrows():
            # Verificar si el usuario existe en el mapeo
            if payment['userid'] in user_mapping:
                soltura_user_id = user_mapping[payment['userid']]
                
                # Mapear el método de pago a un método en Soltura
                payment_method_name = payment['payment_method'] if pd.notna(payment['payment_method']) else "Unknown"
                
                # Buscar método similar en Soltura
                cursor.execute("SELECT PaymentMethodId FROM SocaiPaymentMethods WHERE name LIKE ?", 
                               (f"%{payment_method_name}%",))
                method_result = cursor.fetchone()
                
                if method_result:
                    payment_method_id = method_result[0]
                    
                    # Enmascarar información de la cuenta para mayor seguridad
                    account_info = payment['accountinfo'] if pd.notna(payment['accountinfo']) else "Migrated Payment Method"
                    account_number = payment['accountnumber'] if pd.notna(payment['accountnumber']) else None
                    
                    if account_number:
                        # Enmascarar el número de cuenta para seguridad
                        if len(account_number) > 4:
                            masked_account = "XXXX" + account_number[-4:]
                        else:
                            masked_account = "XXXX"
                    else:
                        masked_account = "XXXX"
                    
                    # Crear token para la tarjeta
                    token = hashlib.sha256(f"{soltura_user_id}_{payment_method_id}_{datetime.now()}".encode()).digest()
                    
                    # Insertar en SocaiDataPayments
                    sql = """
                    INSERT INTO SocaiDataPayments (name, token, expToken, maskAccount, UserId, PaymentMethodId)
                    VALUES (?, ?, ?, ?, ?, ?)
                    """
                    
                    # Fecha de expiración un año después
                    exp_date = datetime.now() + timedelta(days=365)
                    mask_account_binary = masked_account.encode()
                    
                    cursor.execute(sql, (account_info, token, exp_date, mask_account_binary, soltura_user_id, payment_method_id))
                    payment_methods_count += 1
        
        conn.commit()
        print(f"Migración de métodos de pago completa: {payment_methods_count} métodos migrados.")
        return True
    except Exception as e:
        conn.rollback()
        print(f"Error migrando métodos de pago: {e}")
        return False
    finally:
        conn.close()


if __name__ == "__main__":
    main()